## learning-AI101 : insight 4
### U-net (encoder to decoder) 구조를 이용한 number of opening ion channel classification 분석

<br>

- **임규연 (lky473736)**
- 2024.08.16. ~ 2024.08.20. 문서 작성
- **dataset** : https://www.kaggle.com/competitions/liverpool-ion-switching/data?select=test.csv
- **kaggle** : https://www.kaggle.com/competitions/liverpool-ion-switching/data?select=test.csv
- **data abstract** : you will be predicting the number of open_channels present, based on electrophysiological signal data.



------

본 데이터셋에 대하여 설명해보자면, 예전에 "Liverpool Ion Switching"이라는 대회에서 제공하였으며, 이온 채널의 개폐 상태를 예측하는 것이 목표이다. 전기 신호 데이터를 분석하여 특정 시간에 이온 채널이 몇개가 열려 있는지를 예측해야 하며, 입력 데이터는 전기 신호 값인 **'signal'**, 출력 데이터는 0~10개의 **'open_channels'** 이다. 

따라서 이는 **다중 클래스 classification**을 이용하여 해결이 가능하다. 결국엔 0~10까지의 정수를 prediction하는 것일테니, 11개의 클래스 중 하나를 뽑는 분류 문제이다. 

그 당시, 유저 **K_MAT은 이를 U-net 구조와 1D-CNN을 이용하여 구현하**였으며, public score를 0.91까지 끌어올렸다. version 1~4 중 version 4에서의 소스 코드 (https://www.kaggle.com/code/kmat2019/u-net-1d-cnn-with-keras) 를 이 insight에서 분석 후에, 추후 report directory에서 내 방식대로 ML classification, DL classification할 계획이다.

In [3]:
import os
import matplotlib.pyplot as plt
import glob
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.layers import Dense, Dropout, Reshape, Conv1D, BatchNormalization, Activation, AveragePooling1D, GlobalAveragePooling1D, Lambda, Input, Concatenate, Add, UpSampling1D, Multiply
from keras.models import Model
# objectives 작동 X -> losses로 변경
from keras.losses import mean_squared_error
from keras import backend as K
from keras.losses import binary_crossentropy, categorical_crossentropy
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau,LearningRateScheduler
from keras.initializers import random_normal
from keras.optimizers import Adam, RMSprop, SGD
from keras.callbacks import Callback

from sklearn.metrics import cohen_kappa_score, f1_score
from sklearn.model_selection import KFold, train_test_split

-------

### 기존 코드 분석 및 U-net에 대한 정의

- reference : https://joungheekim.github.io/2020/09/28/paper-review/
<br>

- 여기서부터 https://www.kaggle.com/code/kmat2019/u-net-1d-cnn-with-keras 에 소스 코드를 여기에 붙여 넣어서 분석한다.
- 현재 이 코드에서는 1D-CNN을 통해 time-series data를 분석하고 있다. 
- U-net 구조는 **auto-encoder** 기반 모델이며, encoder (down-sampling)와 decoder (up-sampling) 부분으로 나뉘어진다. 
    - encoder를 통해 data의 중요한 feature을 압축한다. 압축된 정보를 latent space라고 일컫는다. (context vector 등 부르는 말은 다양하다.)
    - 중간에 latent space에서 다시 decoder를 통해 원래 사이즈로 복원한다.

<img src='https://i.imgur.com/2MUbGYf.png' width='500px'>

- 왼쪽 영역, **즉 encoder 부분을 contracting path, decoder 부분을 bridge라 부른다.**
- **contracting path의 각 level은 encoder block으로 이루어진다.** encoder block엔 conv block이 속해있다.
    - conv block : conv, relu, batch normalization
    - encoder block : conv block + pooling
    
- **decoder의 각 level은 decoder block으로 이루어진다.**
    - decoder block : convtranspose, concatenate, convblock (encoder와 반대 logic으로, 원래와 같은 크기가 되도록 함)
    
    
<br>

- U-net 구성
    - **encoder**
        - 데이터를 압축하여 important feature를 extraction
        - 연속적인 합성곱과 풀링 연산으로 이루어짐 (CNN에서 합성곱층과 풀링층을 몇개씩 나란히 쓰는 것처럼)
    - **decoder**
        - encoder에서 추출된 feature를 사용하여 원래 해상도로 복원함
        - upsampling, 합성곱 연산으로 복원함
    - **skip connection (latent space)**
        - encoder에서 각 단계에서 추출된 important feature으로 구성된 feature map을 decoder와 대응하는 단계 (latent space)

In [4]:
df_train = pd.read_csv("./data/ion_switching/train.csv")
df_test = pd.read_csv("./data/ion_switching/test.csv")

print (df_train.info())
print ()
print (df_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000000 entries, 0 to 4999999
Data columns (total 3 columns):
 #   Column         Dtype  
---  ------         -----  
 0   time           float64
 1   signal         float64
 2   open_channels  int64  
dtypes: float64(2), int64(1)
memory usage: 114.4 MB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 2 columns):
 #   Column  Dtype  
---  ------  -----  
 0   time    float64
 1   signal  float64
dtypes: float64(2)
memory usage: 30.5 MB
None


- df_train에는 훈련 데이터, df_test에는 테스트 데이터가 저장되어 있다.

In [5]:
train_input = df_train["signal"].values.reshape(-1, 4000, 1)
train_input_mean = train_input.mean()
train_input_sigma = train_input.std()
train_input = (train_input - train_input_mean) / train_input_sigma

test_input = df_test["signal"].values.reshape(-1, 10000, 1)
test_input = (test_input - train_input_mean) / train_input_sigma

- 위 셀에서 reshape를 통해 signal 열을 3D ndarray로 재구성하였다. 여기서 4000은 타임 스텝이다. 이것이 train_input이 된다. (X_train)
- train_input을 정규화한다. 정규화하는 식은 Z-score normalization 방식을 따르고 있으며, 표준편차 및 평균을 구하여 표준정규분포로 정규화하고 있다.
- test input 또한 마찬가지이다.

In [6]:
train_target = pd.get_dummies(df_train["open_channels"]).values.reshape(-1, 4000, 11)

- open_channels 열을 target으로 두었다. 여기서 get_dummies 메소드를 이용하여 one-hot encoding을 진행하였으며, 11개의 클래스를 나타내는 배열로 변환하였다.
- 4000개의 타임 스텝으로 구성하였다.

In [7]:
idx = np.arange(train_input.shape[0])
train_idx, val_idx = train_test_split(idx, 
                                      random_state=111, 
                                      test_size=0.2)

val_input = train_input[val_idx]
train_input = train_input[train_idx]
val_target = train_target[val_idx]
train_target = train_target[train_idx]

print("train_input:{}, val_input:{}, train_target:{}, val_target:{}".format(train_input.shape, val_input.shape, train_target.shape, val_target.shape))

train_input:(1000, 4000, 1), val_input:(250, 4000, 1), train_target:(1000, 4000, 11), val_target:(250, 4000, 11)


- idx로 train_input 크기만큼의 배열을 만들었고, 이를 하는 이유는 train set, test set을 구성할 때 records를 섞어 편향 문제를 방지하기 위함이다.
- train set : val set = 8 : 2

아래부터는 U-net을 구성하기 위한 함수를 정의한다. 간단한 개요를 작성해보자면 아래와 같다.

- cbr : 인코더와 디코더에서 모델의 표현력, 즉 특징 추출 + 비선형성 추가를 돕는 layers를 만드는 함수. 모든 합성곱 연산에서 cbr 사용.
- se_block : Squeeze and Excitation (SE)을 정의하는 함수
- resblock : ResNet을 정의하는 함수
- Unet : 전체 U-Net 모델 정의

In [8]:
def cbr(x, out_layer, kernel, stride, dilation):
    x = Conv1D(out_layer, kernel_size=kernel, dilation_rate=dilation, strides=stride, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

- Conv1D -> BatchNormalization -> relu를 통하여 비선형성을 추가하며, 각 단계에서 important feature를 추출하는 역할을 하고 있다.
- encoder : 중요한 특징 추출 + 추상적인 표현 학습 (pooling) 
- decoder : cbr이 upsampling된 데이터에 적용 + 똑같이 중요한 특징 추출 ...

In [9]:
def se_block(x_in, layer_n):
    x = GlobalAveragePooling1D()(x_in)
    x = Dense(layer_n // 8, activation="relu")(x)
    x = Dense(layer_n, activation="sigmoid")(x)
    x_out = Multiply()([x_in, x])
    return x_out

- reference 1 : https://ech97.tistory.com/entry/seblock
- reference 2 : https://deep-learning-study.tistory.com/539

<br> 

- squeeze and excitation 블록을 정의하고 있다. 각 채널별로 가중치를 부여하여 feature map의 각 채널에 곱한다. 
- squeeze (압축)
     - 각 채널을 1차원으로 만드는 역할이다. (**global average pooling 1D**)
- excitation (재조정)
     - squeeze에서 생성된 1차원 벡터를 normalization하여 가중치를 부여한다.
     - 본 코드에서는 layer_n // 8을 통하여 채널을 축소하였는데, 이때 8은 hyperparamter이다.
- 본 함수는 채널 간 중요도를 학습하여 중요한 채널에 집중할 수 있게끔 돕는다.

In [10]:
def resblock(x_in, layer_n, kernel, dilation, use_se=True):
    x = cbr(x_in, layer_n, kernel, 1, dilation)
    x = cbr(x, layer_n, kernel, 1, dilation)
    if use_se:
        x = se_block(x, layer_n)
    x = Add()([x_in, x])
    return x  

- 두번의 cbr 연산 후의 출력을 입력과 더하고 있다. 이는 아마도 **vanishing gradient problem**을 해결하기 위함이 아닐까 싶다. 
- 왜냐하면, 여러 층을 거친 result를 입력과 더하는 방식이니 결국엔 입력과 이전 정보의 합을 통하여 출력값이 너무 작아지는 문제를 방지할 수 있을 것이다. 
- 결국 skip connection을 통하여 ($y = F(x) + x$) 합을 이룰 것이고, 이를 반환하는 함수이다.
- 역전파 과정에서의 gradient의 flow를 보자.
    - 일반 network 
        - $\frac{\partial L}{\partial x} = \frac{\partial L}{\partial y} \cdot \frac{\partial y}{\partial x} = \frac{\partial L}{\partial F(x)} \cdot \frac{\partial F(x)}{\partial x}$
        - 여기서 L이 의미하는 바는 손실함수인데, 만약 $\frac{\partial F(x)}{\partial x}$이 작다면. 기울기가 매우 작아진다.
    - resblock
        - $\frac{\partial L}{\partial x} = \frac{\partial L}{\partial y} \cdot \frac{\partial y}{\partial x} = \frac{\partial L}{\partial y} \cdot \left( \frac{\partial F(x)}{\partial x} + 1 \right)$
        - 결국에 **1**이라는 상수가 항상 더해져 기울기가 완전히 소실되는 문제를 피하였다.

In [11]:
def Unet(input_shape=(None, 1)):
    layer_n = 64
    kernel_size = 7
    depth = 2

    input_layer = Input(input_shape)    
    input_layer_1 = AveragePooling1D(5)(input_layer)
    input_layer_2 = AveragePooling1D(25)(input_layer)
    
    ########## Encoder
    x = cbr(input_layer, layer_n, kernel_size, 1, 1)
    for i in range(depth):
        x = resblock(x, layer_n, kernel_size, 1)
    out_0 = x

    x = cbr(x, layer_n * 2, kernel_size, 5, 1)
    for i in range(depth):
        x = resblock(x, layer_n * 2, kernel_size, 1)
    out_1 = x

    x = Concatenate()([x, input_layer_1])
    x = cbr(x, layer_n * 3, kernel_size, 5, 1)
    for i in range(depth):
        x = resblock(x, layer_n * 3, kernel_size, 1)
    out_2 = x

    x = Concatenate()([x, input_layer_2])
    x = cbr(x, layer_n * 4, kernel_size, 5, 1)
    for i in range(depth):
        x = resblock(x, layer_n * 4, kernel_size, 1)
    
    ########### Decoder
    x = UpSampling1D(5)(x)
    x = Concatenate()([x, out_2])
    x = cbr(x, layer_n * 3, kernel_size, 1, 1)

    x = UpSampling1D(5)(x)
    x = Concatenate()([x, out_1])
    x = cbr(x, layer_n * 2, kernel_size, 1, 1)

    x = UpSampling1D(5)(x)
    x = Concatenate()([x, out_0])
    x = cbr(x, layer_n, kernel_size, 1, 1)    

    # Classifier Layer
    x = Conv1D(11, kernel_size=kernel_size, strides=1, padding="same")(x)
    out = Activation("softmax")(x)
    
    model = Model(input_layer, out)
    
    return model

- 위 함수에서 U-Net 모델 자체를 정의한다. 
- encoder 
    - cbr과 resblock을 사용하여 깊은 층에서 feature learning
    - 각 단계에서의 중간 feature map인 out_0, out_1, out_2를 따로 변수로 빼서, decoder에서 사용한다. 
- decoder
    - **UpSampling1D**를 사용하여 차원을 확장하였다. 그리고 out_0, out_1, out_2와 결합하였다.
    - 결합한 걸 다시 cbr하여 이전의 정보를 복원하였다.
- 마지막엔 softmax activation function으로 각 클래스에 대하여 확률을 계산하였다. (나중에 argmax을 통한 연산이 나올 것이라고 예상해본다.)

이 이후에는 데이터 증강, generater, macroF1 클래스, fitting 함수, lrs가 작성되었다.

In [12]:
def augmentations(input_data, target_data):
    #flip
    if np.random.rand() < 0.5:    
        input_data = input_data[::-1]
        target_data = target_data[::-1]

    return input_data, target_data

- data augmentation을 진행한다. 보통은 이미지 학습에서 사용하는 tech인줄 알았는데, 이런 time-series에서 사용할 줄은 몰랐다.
- 입력 데이터를 random.rand()을 이용하여 뒤집는다. 데이터를 변형하여 학습을 더욱 강화하는 것이다.

In [13]:
def Datagen(input_dataset, target_dataset, batch_size, is_train=False):
    x = []
    y = []
    count = 0
    idx_1 = np.arange(len(input_dataset))
    np.random.shuffle(idx_1)

    while True:
        for i in range(len(input_dataset)):
            input_data = input_dataset[idx_1[i]]
            target_data = target_dataset[idx_1[i]]

            if is_train:
                input_data, target_data = augmentations(input_data, target_data)
                
            x.append(input_data)
            y.append(target_data)
            count += 1
            
            if count == batch_size:
                x=np.array(x, dtype=np.float32)
                y=np.array(y, dtype=np.float32)
                inputs = x
                targets = y       
                x = []
                y = []
                count=0
                yield inputs, targets

- 위 함수는 배치 단위로 데이터를 생성하는 generater의 역할을 하고 있다. input_data, target_data를 augmentations 함수를 사용하여 데이터 증강을 적용하고 있다.
- 지정된 배치 단위로 ndarray를 만들어 inputs과 targets를 반환한다.

In [14]:
class macroF1(Callback):
    def __init__(self, model, inputs, targets):
        self.model = model
        self.inputs = inputs
        self.targets = np.argmax(targets, axis=2).reshape(-1)

    def on_epoch_end(self, epoch, logs):
        # 각 에포크가 끝날 때마다 검증 데이터로 매크로 F1 스코어를 계산하여 출력
        pred = np.argmax(self.model.predict(self.inputs), axis=2).reshape(-1)
        f1_val = f1_score(self.targets, pred, average="macro")
        print("val_f1_macro_score: ", f1_val)

- 케라스의 내장 클래스인 **Callback**을 상속받은 것을 확인 가능하며, 이는 model fitting 중에 validation data를 이용하여 f1 score을 계산하여 출력하는 것을 확인 가능하다.
- 역시나 argmax을 통하여 가장 높은 확률을 가진 class를 pred에 넣고, f1_score 메소드를 이용해 f1 score을 측정한다.

In [15]:
def model_fit(model, train_inputs, train_targets, val_inputs, val_targets, n_epoch, batch_size=32):
    # 모델을 학습시키기 위한 함수
    hist = model.fit_generator(
        Datagen(train_inputs, train_targets, batch_size, is_train=True),
        steps_per_epoch=len(train_inputs) // batch_size,
        epochs=n_epoch,
        validation_data=Datagen(val_inputs, val_targets, batch_size),
        validation_steps=len(val_inputs) // batch_size,
        callbacks=[lr_schedule, macroF1(model, val_inputs, val_targets)],
        shuffle=False,
        verbose=1
    )
    return hist


- 말 그대로, 모델을 학습한다.
- fit_generator를 두어 큰 데이터에 대한 학습이 용이하게 하였다. 학습 중간에 학습율 조정을 위해 lr_schedule callback을 두었다.
    - **fit_generator는 사실 권장되는 방법이 아니다.** tensorflow 2.5.0 버전까지만 사용이 가능하며, 2.6.0 이상에서 권장되는 방법은 그저 fit 메소드 하나다.
    - 현재 사용하는 tensorflow 버전은 2.4.0이라 본 코드가 동작되는 것이다.
- macroF1 callback을 통하여 매 학습 시 f1 score 계산을 수행한다.

In [16]:
def lrs(epoch):
    if epoch<35:
        lr = learning_rate
    elif epoch<50:
        lr = learning_rate/10
    else:
        lr = learning_rate/100
    return lr

- lrs를 통하여 학습율을 조정한다. epoch에 따라서 learning_rate에 연산을 가해 조정하고 있다.

In [20]:
K.clear_session()
model = Unet()
#print(model.summary())

learning_rate=0.0005
n_epoch=60
batch_size=32

lr_schedule = LearningRateScheduler(lrs)

#regressor
#model.compile(loss="mean_squared_error", 
#              optimizer=Adam(lr=learning_rate),
#              metrics=["mean_absolute_error"])

#classifier
model.compile(loss=categorical_crossentropy, 
              optimizer=Adam(lr=learning_rate), 
              metrics=["accuracy"])

hist = model_fit(model, train_input, train_target, val_input, val_target, n_epoch, batch_size)

Epoch 1/60


/var/folders/_z/gryfr07n59jgb3wrd062h1ym0000gn/T/ipykernel_35766/3759982232.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = model.fit_generator(
2024-08-20 19:43:11.477930: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


31/31 [==============================] - ETA: 0s - loss: 1.4860 - accuracy: 0.4641

2024-08-20 19:44:44.328110: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


8/8 [==============================] - 6s 552ms/step
val_f1_macro_score:  0.0360673573788344
31/31 [==============================] - 104s 3s/step - loss: 1.4860 - accuracy: 0.4641 - val_loss: 2.6236 - val_accuracy: 0.1959 - lr: 5.0000e-04
Epoch 2/60
8/8 [==============================] - 6s 691ms/steposs: 1.0791 - 
val_f1_macro_score:  0.08903828171990068
31/31 [==============================] - 100s 3s/step - loss: 1.0791 - accuracy: 0.5877 - val_loss: 7.5935 - val_accuracy: 0.3069 - lr: 5.0000e-04
Epoch 3/60
8/8 [==============================] - 5s 616ms/steposs: 0.9192 - 
val_f1_macro_score:  0.03463572660246789
31/31 [==============================] - 91s 3s/step - loss: 0.9192 - accuracy: 0.6405 - val_loss: 13.9496 - val_accuracy: 0.2243 - lr: 5.0000e-04
Epoch 4/60
8/8 [==============================] - 5s 595ms/steposs: 0.8869 - 
val_f1_macro_score:  0.035160121010793015
31/31 [==============================] - 96s 3s/step - loss: 0.8869 - accuracy: 0.6471 - val_loss: 19.3717 -

Epoch 32/60
8/8 [==============================] - 4s 522ms/steposs: 0.2214 - 
val_f1_macro_score:  0.6527946917007532
31/31 [==============================] - 79s 3s/step - loss: 0.2214 - accuracy: 0.9247 - val_loss: 0.6331 - val_accuracy: 0.7893 - lr: 5.0000e-04
Epoch 33/60
8/8 [==============================] - 4s 512ms/steposs: 0.2311 - 
val_f1_macro_score:  0.8517661516084584
31/31 [==============================] - 81s 3s/step - loss: 0.2311 - accuracy: 0.9202 - val_loss: 0.2847 - val_accuracy: 0.8957 - lr: 5.0000e-04
Epoch 34/60
8/8 [==============================] - 4s 519ms/steposs: 0.5017 - 
val_f1_macro_score:  0.21008941385935181
31/31 [==============================] - 79s 3s/step - loss: 0.5017 - accuracy: 0.8339 - val_loss: 19.5369 - val_accuracy: 0.4786 - lr: 5.0000e-04
Epoch 35/60
8/8 [==============================] - 4s 514ms/steposs: 0.3556 - 
val_f1_macro_score:  0.48222247370521126
31/31 [==============================] - 79s 3s/step - loss: 0.3556 - accuracy: 0.8

In [21]:
pred = np.argmax((model.predict(val_input)+model.predict(val_input[:,::-1,:])[:,::-1,:])/2, axis=2).reshape(-1)
gt = np.argmax(val_target, axis=2).reshape(-1)
print("SCORE_oldmetric: ", cohen_kappa_score(gt, pred, weights="quadratic"))
print("SCORE_newmetric: ", f1_score(gt, pred, average="macro"))

8/8 [==============================] - 4s 541ms/step
SCORE_oldmetric:  0.9941988168308229
SCORE_newmetric:  0.8845282496397201
